In [1]:
import pandas as pd
from stage import fullPrint
from datetime import datetime, timedelta
import yahoo_fin.stock_info as yf
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)

In [2]:
allList = []
nasdaqList = yf.tickers_nasdaq()
allList = (nasdaqList + nyseList)

In [3]:
SectorDict = {
    "Energy":"XLE",
    "Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Defensive":"XLP",
    "Healthcare":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrials":"XLI",
    "Utilities":"XLU",
    "Basic Materials":"XLB",
    "Financial Services":"XLF",
    "Real Estate":"XLRE"}
spList = yf.tickers_sp500()
spList = set(spList)
dictionary = {}
i = 0
allLen = str(len(nasdaqList))

now = datetime.now()
monday = now - timedelta(days = now.weekday())
monday = monday.strftime('%Y-%m-%d')

for ticker in nasdaqList:
    try:
        df = yf.get_data(ticker, start_date="1995-01-02",end_date = monday, index_as_date = True, interval="1wk")
        df.to_pickle('stockData/nysenasdaq/'+ticker+'.pkl')
        industry = yf.get_company_info(ticker).at['sector','Value']
        dictionary[ticker] = SectorDict[industry]
    except:
        print(ticker+" not found")
    i = i+1
    print(str(i)+'/'+allLen)
import pickle
try:
    sectorDict = open('stockData/nasdaq.pkl', 'wb')
    pickle.dump(dictionary, sectorDict)
    sectorDict.close()
  
except:
    print("Something went wrong")

1/5574
2/5574
3/5574
AACIW not found
4/5574
5/5574
AADR not found
6/5574
7/5574
8/5574
9/5574
10/5574
11/5574
12/5574
13/5574
AAXJ not found
14/5574
15/5574
16/5574
17/5574
18/5574
19/5574
20/5574
21/5574
22/5574
23/5574
24/5574
25/5574
26/5574
27/5574
28/5574
ACABU not found
29/5574
30/5574
31/5574
32/5574
ACAHW not found
33/5574
ACAXU not found
34/5574
35/5574
36/5574
37/5574
ACBAW not found
38/5574
39/5574
40/5574
41/5574
42/5574
43/5574
44/5574
ACEVW not found
45/5574
46/5574
47/5574
48/5574
49/5574
50/5574
51/5574
52/5574
53/5574
ACKIT not found
54/5574
ACKIU not found
55/5574
ACKIW not found
56/5574
57/5574
ACLX not found
58/5574
59/5574
60/5574
61/5574
62/5574
63/5574
ACQRW not found
64/5574
65/5574
66/5574
67/5574
68/5574
69/5574
70/5574
ACTDW not found
71/5574
72/5574
73/5574
ACWI not found
74/5574
ACWX not found
75/5574
76/5574
77/5574
78/5574
79/5574
80/5574
81/5574
82/5574
83/5574
84/5574
ADERW not found
85/5574
86/5574
87/5574
88/5574
89/5574
ADILW not found
90/5574
91/557

In [ ]:
import numpy as np
import yahoo_fin.stock_info as yf
weights = np.arange(1,31)
sumWeights = np.sum(weights)
spList = yf.tickers_sp500()
def calculateSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,4]/df.iat[index-1,4]

def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iat[index,0]/df.iat[index-30,0]
def calculatePeak(df,index, diff, col):
    if index == 0:
        return df.iat[index,0]
    if index < diff:
        return max(df.iat[index-1,col],df.iat[index,0])


In [ ]:
from stage import fullPrint
import swifter
import yahoo_fin.stock_info as yf
import pandas as pd
import os
spDf = yf.get_data('^GSPC', start_date="1995-01-02",end_date = monday, index_as_date = True, interval="1wk")
for ticker in allList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        if(stockDf['close'].isnull().values.any()):
            os.remove('stockData/nyseNasdaq/'+ticker+'.pkl')
            continue
        stockDf['fiveYearHigh'] = stockDf['close'].rolling(window=260).max().shift(1).fillna(0)  
        stockDf['WMA30'] = stockDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
        stockDf['WMA30Slope'] = stockDf.swifter.apply(lambda x: calculateSlopeOfTicker(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['volumePerc'] = stockDf['volume'].pct_change()
        stockDf['percent'] = stockDf.swifter.apply(lambda x: product(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        spDf['percent'] = spDf.swifter.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
        stockDf['RS'] = stockDf['percent'] - spDf['percent']
        #stockDf = stockDf.dropna()
        stockDf['peak'] = 0
        stockDf['trough'] = 0
        stockDf['support'] = 0
        stockDf['initialSupport'] = 0
        for index, element in stockDf.iterrows():
            numIndex =stockDf.index.get_loc(index)
            if numIndex < 260:
                stockDf.iat[numIndex, 3] = calculatePeak(stockDf, numIndex, 260, 3)
        stockDf['Stage'] = ""
        stockDf['secondBuy'] = False
        stockDf['resistance'] = 0
        stockDf['shortPeak'] = 0
        stockDf['shortTrough'] = 0
        print(ticker)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass

Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 9957.30it/s]


AACG


Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 9754.45it/s]

AACI

Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 10524.68it/s]


...

ZUO


Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 9766.38it/s]


ZWS


Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 13156.30it/s]


ZYME


In [ ]:
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-02",end_date= monday, index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [ ]:
pd.read_pickle('stockData/nyseNasdaq/AAPL.pkl').to_csv("AAPL.csv")

In [ ]:
for ticker in allList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        stockDf = stockDf.drop(['open','high','low','adjclose'],axis=1)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass